In [1]:
import pandas as pd

In [7]:
test = pd.read_json('/Volumes/SecondDrive/ahr_data.jsonl', lines=True)

In [10]:
test[test.fullText.isna()==False][0:1].url.values

array(['http://www.jstor.org/stable/1836876'], dtype=object)

In [2]:
from execeptional_metadata import *
members_df, books_df, events_df = get_shxco_exceptional_data()

/Users/EZCorp/workspace/unknown_borrowers/unknown_borrowers_env/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [13]:
events_df.columns
events_df['start_datetime'] = pd.to_datetime(events_df.start_date, format='%Y-%m-%d', errors='coerce')
events_df['end_datetime'] = pd.to_datetime(events_df.end_date, format='%Y-%m-%d', errors='coerce')
events_df['subscription_purchase_datetime'] = pd.to_datetime(events_df.subscription_purchase_date, format='%Y-%m-%d', errors='coerce')

In [14]:
borrow_events = events_df[(events_df.event_type == 'Borrow') &(events_df.start_date.str.len() > 9) & (events_df.end_date.str.len() > 9)]
borrow_events.loc[:,'year'] = borrow_events.start_datetime.dt.year
borrow_events.loc[:,'month'] = borrow_events.start_datetime.dt.month
borrow_events.columns

Index(['event_type', 'start_date', 'end_date', 'member_uris', 'member_names',
       'member_sort_names', 'subscription_price_paid', 'subscription_deposit',
       'subscription_duration', 'subscription_duration_days',
       'subscription_volumes', 'subscription_category',
       'subscription_purchase_date', 'reimbursement_refund', 'borrow_status',
       'borrow_duration_days', 'purchase_price', 'currency', 'item_uri',
       'item_title', 'item_volume', 'item_authors', 'item_year', 'item_notes',
       'source_type', 'source_citation', 'source_manifest', 'source_image',
       'first_member_uri', 'second_member_uri', 'member_id', 'index_col', 'dt',
       'exceptional_types', 'books_out', 'excess_books_out', 'start_datetime',
       'end_datetime', 'subscription_purchase_datetime', 'year', 'month'],
      dtype='object')

In [17]:
subscription_events = events_df.copy()
subscription_events = subscription_events[subscription_events.event_type.isin(['Subscription', 'Renewal', 'Supplement'])]

subset_subscription_events = subscription_events[['subscription_purchase_datetime','start_datetime', 'event_type', 'member_names', 'end_datetime', 'index_col', 'first_member_uri','second_member_uri']]
member_subs = pd.merge(left=subset_subscription_events, right=members_df, left_on="first_member_uri", right_on="uri")
member_subs = member_subs[member_subs.has_card]
member_subs

,subscription_purchase_datetime,start_datetime,event_type,member_names,end_datetime,index_col,first_member_uri,second_member_uri,uri,name,...,postal_codes,arrondissements,coordinates,notes,updated,member_id,borrow_count,subscription_count,exceptional_types,exceptional_counts
20,1919-11-22,1919-11-22,Subscription,Marie Boscq / Bosq,1920-11-22,7,https://shakespeareandco.princeton.edu/members...,NaN,https://shakespeareandco.princeton.edu/members...,Marie Boscq / Bosq,...,75006,6,"48.85025, 2.33820",NaN,2021-01-06T21:44:22+00:00,boscq-marie,0,10,"overborrows,post1942_events","18,1"
21,1920-12-31,1920-12-31,Subscription,Marie Boscq / Bosq,1921-06-30,266,https://shakespeareandco.princeton.edu/members...,NaN,https://shakespeareandco.princeton.edu/members...,Marie Boscq / Bosq,...,75006,6,"48.85025, 2.33820",NaN,2021-01-06T21:44:22+00:00,boscq-marie,0,10,"overborrows,post1942_events","18,1"
22,1921-08-25,1921-08-25,Renewal,Marie Boscq / Bosq,1922-02-25,491,https://shakespeareandco.princeton.edu/members...,NaN,https://shakespeareandco.princeton.edu/members...,Marie Boscq / Bosq,...,75006,6,"48.85025, 2.33820",NaN,2021-01-06T21:44:22+00:00,boscq-marie,0,10,"overborrows,post1942_events","18,1"
23,1932-10-15,1932-10-15,Subscription,Marie Boscq / Bosq,1933-04-14,18886,https://shakespeareandco.princeton.edu/members...,NaN,https://shakespeareandco.princeton.edu/members...,Marie Boscq / Bosq,...,75006,6,"48.85025, 2.33820",NaN,2021-01-06T21:44:22+00:00,boscq-marie,0,10,"overborrows,post1942_events","18,1"
24,1933-05-15,1933-04-14,Renewal,Marie Boscq / Bosq,1934-04-14,19446,https://shakespeareandco.princeton.edu/members...,NaN,https://shakespeareandco.princeton.edu/members...,Marie Boscq / Bosq,...,75006,6,"48.85025, 2.33820",NaN,2021-01-06T21:44:22+00:00,boscq-marie,0,10,"overborrows,post1942_events","18,1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,1941-11-28,1941-11-28,Subscription,Jean Pierret,1941-12-28,34544,https://shakespeareandco.princeton.edu/members...,NaN,https://shakespeareandco.princeton.edu/members...,Jean Pierret,...,75014,14,"48.82993, 2.32034",NaN,2019-11-19T14:47:36+00:00,pierret,0,1,NaN,NaN
9644,1941-11-28,1941-11-28,Subscription,Robert Gillet,1941-12-28,34545,https://shakespeareandco.princeton.edu/members...,NaN,https://shakespeareandco.princeton.edu/members...,Robert Gillet,...,75017,17,"48.88551, 2.30273",NaN,2020-01-02T20:45:04+00:00,gillet-robert,0,1,unknown_borrows,1
9645,1941-11-28,1941-11-28,Subscription,Bernard Lestocquoy,1941-12-28,34546,https://shakespeareandco.princeton.edu/members...,NaN,https://shakespeareandco.princeton.edu/members...,Bernard Lestocquoy,...,75116,16,"48.86765, 2.28811",NaN,2019-11-19T14:47:34+00:00,lestocquoy,0,1,unknown_borrows,1
9646,1941-11-28,1941-11-28,Subscription,Christianne Mingalon,1941-12-28,34548,https://shakespeareandco.princeton.edu/members...,NaN,https://shakespeareandco.princeton.edu/members...,Christianne Mingalon,...,75004,4,"48.85995, 2.35299",NaN,2019-11-19T14:47:35+00:00,mingalon,0,1,NaN,NaN


In [147]:
original_events = pd.read_csv('./data/SCoData_events_v1.1_2021-01.csv')

original_events['index_col'] = original_events.index
len(original_events)

35031

In [155]:
events_df = pd.read_csv('SCoData_events_v1.1_2021-01_exceptional.csv')
len(events_df)

34357

In [149]:
m = original_events.index_col.isin(events_df.index_col)
original_events[~m]

,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,...,item_title,item_volume,item_authors,item_year,item_notes,source_type,source_citation,source_manifest,source_image,index_col
297,Renewal,1921-02-09,1921-03-09,https://shakespeareandco.princeton.edu/members...,Marjory Gibb / Mrs. Phelan Gibb;Phelan Gibb / ...,"Gibb, Marjory;Gibb, Phelan",8.0,NaN,1 month,28.0,...,NaN,NaN,NaN,NaN,NaN,Logbook,"Sylvia Beach, Logbooks 1919–1941, Sylvia Beach...",NaN,NaN,297
374,Subscription,1921-04-23,1921-10-23,https://shakespeareandco.princeton.edu/members...,Mr. Kershaw;Mrs. Kershaw,"Kershaw, Mr.;Kershaw, Mrs.",35.0,7.0,6 months,183.0,...,NaN,NaN,NaN,NaN,NaN,Logbook,"Sylvia Beach, Logbooks 1919–1941, Sylvia Beach...",NaN,NaN,374
480,Subscription,1921-08-05,1921-11-05,https://shakespeareandco.princeton.edu/members...,Mr. Wrenn;Mrs. Wrenn,"Wrenn, Mr.;Wrenn, Mrs.",20.0,7.0,3 months,92.0,...,NaN,NaN,NaN,NaN,NaN,Logbook,"Sylvia Beach, Logbooks 1919–1941, Sylvia Beach...",NaN,NaN,480
497,Subscription,1921-08-30,1921-11-30,https://shakespeareandco.princeton.edu/members...,M. Tessier;Mme Tessier,"Tessier, M.;Tessier, Mme",20.0,14.0,3 months,92.0,...,NaN,NaN,NaN,NaN,NaN,Logbook,"Sylvia Beach, Logbooks 1919–1941, Sylvia Beach...",NaN,NaN,497
585,Subscription,1921-11-08,1922-02-08,https://shakespeareandco.princeton.edu/members...,Mr. Wallner;Mrs. Wallner,"Wallner, Mr.;Wallner, Mrs.",30.0,14.0,3 months,92.0,...,NaN,NaN,NaN,NaN,NaN,Logbook,"Sylvia Beach, Logbooks 1919–1941, Sylvia Beach...",NaN,NaN,585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30840,Borrow,1939-07-04,1939-10-05,https://shakespeareandco.princeton.edu/members...,Mesures,Mesures,NaN,NaN,NaN,NaN,...,Heaven's My Destination,NaN,"Wilder, Thornton",1934.0,NaN,Lending Library Card,"Sylvia Beach, Mesures Lending Library Card, Bo...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/1a...,30840
30841,Borrow,1939-07-04,1939-10-05,https://shakespeareandco.princeton.edu/members...,Mesures,Mesures,NaN,NaN,NaN,NaN,...,The Cabala,NaN,"Wilder, Thornton",1926.0,NaN,Lending Library Card,"Sylvia Beach, Mesures Lending Library Card, Bo...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/1a...,30841
30842,Borrow,1939-07-04,1939-10-05,https://shakespeareandco.princeton.edu/members...,Mesures,Mesures,NaN,NaN,NaN,NaN,...,The Woman of Andros,NaN,"Wilder, Thornton",1930.0,NaN,Lending Library Card,"Sylvia Beach, Mesures Lending Library Card, Bo...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/1a...,30842
30843,Borrow,1939-07-04,1939-10-05,https://shakespeareandco.princeton.edu/members...,Mesures,Mesures,NaN,NaN,NaN,NaN,...,The Bridge of San Luis Rey,NaN,"Wilder, Thornton",1927.0,NaN,Lending Library Card,"Sylvia Beach, Mesures Lending Library Card, Bo...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/1a...,30843


In [150]:
original_events[
        ["first_member_uri", "second_member_uri"]
    ] = original_events.member_uris.str.split(";", expand=True)
original_events["member_id"] = original_events.first_member_uri.apply(
        lambda x: x.split("/")[-2]
    )
original_events["item_uri"] = original_events.item_uri.apply(
        lambda x: x.split("/")[-2] if pd.notna(x) else None
    )

In [146]:
test_df = pd.read_csv('test_subset.csv')
len(test_df)
# test_df[['index_col', 'item_uri', 'member_uris', 'start_date', 'exceptional_types', 'books_out', 'excess_books_out', 'event_type', 'first_member_uri', 'second_member_uri']]

7627

In [153]:
test2 = pd.merge(original_events, test_df[['index_col', 'books_out', 'excess_books_out', 'event_type']], on=['index_col'], how='left')
len(test2)

35731

In [154]:
test2 = test2.drop_duplicates(subset='index_col')
len(test2)

35031

In [117]:
len(original_events)

35031

In [137]:
m = test2.index_col.isin(original_events.index_col)
test2[~m]

,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,...,source_citation,source_manifest,source_image,index_col,first_member_uri,second_member_uri,member_id,exceptional_types,books_out,excess_books_out


In [141]:
len(test2.index_col.unique().tolist()), len(test2)

(35031, 35055)

In [142]:
test2[test2.index_col.duplicated()]

,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,...,source_citation,source_manifest,source_image,index_col,first_member_uri,second_member_uri,member_id,exceptional_types,books_out,excess_books_out
13882,Borrow,1927-02-09,1928-02-15,https://shakespeareandco.princeton.edu/members...,Pierre Vogein,"Vogein, Pierre",NaN,NaN,NaN,NaN,...,"Sylvia Beach, Pierre Vogein Lending Library Ca...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif-cloud.princeton.edu/iiif/2/af%2Fc...,13827,https://shakespeareandco.princeton.edu/members...,None,vogein,"overborrows,longterm_borrows",NaN,NaN
14945,Borrow,1928-03-28,1929-05-08,https://shakespeareandco.princeton.edu/members...,Colette Ottensooser,"Ottensooser, Colette",NaN,NaN,NaN,NaN,...,"Sylvia Beach, Colette Ottensooser Lending Libr...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/dc...,14884,https://shakespeareandco.princeton.edu/members...,None,ottensooser,"overborrows,longterm_borrows",NaN,NaN
15188,Borrow,1928-07-09,1929-10-22,https://shakespeareandco.princeton.edu/members...,Colette Ottensooser,"Ottensooser, Colette",NaN,NaN,NaN,NaN,...,"Sylvia Beach, Colette Ottensooser Lending Libr...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/dc...,15123,https://shakespeareandco.princeton.edu/members...,None,ottensooser,"overborrows,longterm_borrows",NaN,NaN
15500,Borrow,1928-11-22,1929-12-03,https://shakespeareandco.princeton.edu/members...,Jeanne Fournier,"Fournier, Jeanne",NaN,NaN,NaN,NaN,...,"Sylvia Beach, Jeanne Fournier Lending Library ...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif-cloud.princeton.edu/iiif/2/a6%2F8...,15432,https://shakespeareandco.princeton.edu/members...,None,fournier-jeanne,"overborrows,longterm_borrows",NaN,NaN
16625,Borrow,1929-10-26,1931-11-14,https://shakespeareandco.princeton.edu/members...,Anatole Rivoallan,"Rivoallan, Anatole",NaN,NaN,NaN,NaN,...,"Sylvia Beach, Rivoallan Lending Library Card, ...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif-cloud.princeton.edu/iiif/2/e0%2F1...,16548,https://shakespeareandco.princeton.edu/members...,None,rivoallan-anatole,"overborrows,longterm_borrows",NaN,NaN
16683,Borrow,1929-11-14,1931-12-21,https://shakespeareandco.princeton.edu/members...,Anatole Rivoallan,"Rivoallan, Anatole",NaN,NaN,NaN,NaN,...,"Sylvia Beach, Rivoallan Lending Library Card, ...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif-cloud.princeton.edu/iiif/2/e0%2F1...,16602,https://shakespeareandco.princeton.edu/members...,None,rivoallan-anatole,"overborrows,longterm_borrows",NaN,NaN
16793,Borrow,1929-12-21,1931-11-19,https://shakespeareandco.princeton.edu/members...,Anatole Rivoallan,"Rivoallan, Anatole",NaN,NaN,NaN,NaN,...,"Sylvia Beach, Rivoallan Lending Library Card, ...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif-cloud.princeton.edu/iiif/2/e0%2F1...,16709,https://shakespeareandco.princeton.edu/members...,None,rivoallan-anatole,"overborrows,longterm_borrows",NaN,NaN
24322,Borrow,1936-05-09,1936-05-19,https://shakespeareandco.princeton.edu/members...,Eleanor (Hayden) Kittredge / Mrs. Tracy B. Kit...,"Kittredge, Eleanor",NaN,NaN,NaN,NaN,...,"Sylvia Beach, Eleanor Hayden Kittredge Lending...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif-cloud.princeton.edu/iiif/2/d3%2F9...,24208,https://shakespeareandco.princeton.edu/members...,None,kittredge-eleanor-hayden,"unknown_borrows,overborrows",4.0,2.0
26212,Borrow,1937-05-18,1937-11-05,https://shakespeareandco.princeton.edu/members...,Françoise de Marcilly,"de Marcilly, Françoise",NaN,NaN,NaN,NaN,...,"Sylvia Beach, Mme de Marcilly Lending Library ...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif-cloud.princeton.edu/iiif/2/7d%2F2...,26088,https://shakespeareandco.princeton.edu/members...,N

In [31]:
events_df['uri'] = events_df.uri.apply(
        lambda x: x.split("/")[-2] if pd.notna(x) else None
    )
events_df

,uri,title,author,translator,editor,contributor,illustrator,introduction,preface,photographer,...,format,uncertain,ebook_url,volumes_issues,notes,event_count,borrow_count,purchase_count,circulation_years,updated
0,malory-morte-darthur,Le Morte d'Arthur,"Malory, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Book,False,NaN,NaN,NaN,7,7,0,1937;1930;1939;1932;1938,2020-06-29T03:50:09+00:00
1,more-utopia,Utopia,"More, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Book,False,NaN,NaN,NaN,2,2,0,1930;1950;1951,2020-04-30T19:40:14+00:00
2,norton-gorboduc,Gorboduc,"Norton, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Book,False,NaN,NaN,NaN,1,1,0,1950,2020-06-29T03:49:47+00:00
3,lyly-euphues-anatomy-wit,Euphues: The Anatomy of Wit,"Lyly, John",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Book,False,NaN,NaN,NaN,1,1,0,1941,2020-04-30T19:40:21+00:00
4,spenser-shepheards-calendar,The Shepheard's Calendar,"Spenser, Edmund",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Book,False,NaN,NaN,NaN,1,1,0,1941,2020-06-29T03:45:41+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6013,unclear-ile,[unclear]ile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,NaN,Unidentified. Handwriting unclear.,1,1,0,NaN,2020-06-05T17:04:58+00:00
6014,unclear-ism,[unclear]ism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,NaN,Unidentified. Handwriting unclear.,1,1,0,1928,2020-06-02T22:08:10+00:00
6015,unclear-4,[unclear]y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,NaN,Unidentified. Handwriting unclear.,1,1,0,1926,2020-08-17T19:32:28+00:00
6016,unidentified-benn-s-library,[unidentified Benn’s Library],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,NaN,Unidentified. From Benn's Sixpenny Library ser...,6,0,6,1931,2020-06-02T22:10:06+00:00


In [33]:
len(events_df.uri.unique().tolist())

6018

In [27]:
events_df['item_uri_updated'] = events_df.uri.apply(
        lambda x: x.split("/")[-2] if pd.notna(x) else None
    )

In [23]:
events_df[(events_df.item_uri.isna() == False) & (events_df.item_uri_updated.isna() == True)]

,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,...,item_title,item_volume,item_authors,item_year,item_notes,source_type,source_citation,source_manifest,source_image,item_uri_updated


In [159]:
events_df[events_df.books_out.isna()==False]

,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,...,source_manifest,source_image,first_member_uri,second_member_uri,member_id,index_col,dt,exceptional_types,books_out,excess_books_out
7987,Borrow,1919-12-04,1919-12-13,https://shakespeareandco.princeton.edu/members...,Raymonde Linossier,"Linossier, Raymonde",NaN,NaN,NaN,NaN,...,https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e0...,https://shakespeareandco.princeton.edu/members...,NaN,linossier-raymonde,8030,1919-12-04,overborrows,2.0,1.0
7988,Borrow,1919-12-04,1919-12-13,https://shakespeareandco.princeton.edu/members...,Raymonde Linossier,"Linossier, Raymonde",NaN,NaN,NaN,NaN,...,https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e0...,https://shakespeareandco.princeton.edu/members...,NaN,linossier-raymonde,8031,1919-12-04,overborrows,2.0,1.0
7993,Borrow,1919-12-13,1919-12-15,https://shakespeareandco.princeton.edu/members...,Raymonde Linossier,"Linossier, Raymonde",NaN,NaN,NaN,NaN,...,https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e0...,https://shakespeareandco.princeton.edu/members...,NaN,linossier-raymonde,8036,1919-12-13,overborrows,3.0,2.0
7994,Borrow,1919-12-13,1919-12-15,https://shakespeareandco.princeton.edu/members...,Raymonde Linossier,"Linossier, Raymonde",NaN,NaN,NaN,NaN,...,https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e0...,https://shakespeareandco.princeton.edu/members...,NaN,linossier-raymonde,8037,1919-12-13,overborrows,3.0,2.0
7995,Borrow,1919-12-13,1919-12-15,https://shakespeareandco.princeton.edu/members...,Raymonde Linossier,"Linossier, Raymonde",NaN,NaN,NaN,NaN,...,https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e0...,https://shakespeareandco.princeton.edu/members...,NaN,linossier-raymonde,8038,1919-12-13,overborrows,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33939,Borrow,1941-12-10,1942-01-12,https://shakespeareandco.princeton.edu/members...,Mlle Mantoy,"Mantoy, Mlle",NaN,NaN,NaN,NaN,...,https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/7f...,https://shakespeareandco.princeton.edu/members...,NaN,mantoy,34613,1941-12-10,overborrows,2.0,1.0
33944,Borrow,1941-12-10,1942-06-25,https://shakespeareandco.princeton.edu/members...,Marie Boscq / Bosq,"Boscq, Marie",NaN,NaN,NaN,NaN,...,https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/53...,https://shakespeareandco.princeton.edu/members...,NaN,boscq-marie,34618,1941-12-10,overborrows,4.0,2.0
33956,Borrow,1941-12-12,1941-12-13,https://shakespeareandco.princeton.edu/members...,R. F. Collins,"Collins, R. F.",NaN,NaN,NaN,NaN,...,https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/03...,https://shakespeareandco.princeton.edu/members...,NaN,collins-r-f,34630,1941-12-12,overborrows,2.0,1.0
33957,Borrow,1941-12-12,1941-12-13,https://shakespeareandco.princeton.edu/members...,R. F. Collins,"Collins, R. F.",NaN,NaN,NaN,NaN,...,https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/03...,https://shakespeareandco.princeton.edu/members...,NaN,collins-r-f,34631,1941-12-12,overborrows,2.0,1.0


In [160]:
members_df = pd.read_csv('SCoData_members_v1.1_2021-01_exceptional.csv')
members_df

,uri,name,sort_name,title,gender,is_organization,has_card,birth_year,death_year,membership_years,...,postal_codes,arrondissements,coordinates,notes,updated,member_id,borrow_count,subscription_count,exceptional_types,exceptional_counts
0,https://shakespeareandco.princeton.edu/members...,"""Friend of Renoir""","""Friend of Renoir""",NaN,NaN,False,False,NaN,NaN,1920;1921,...,NaN,NaN,NaN,NaN,2020-11-04T16:09:29+00:00,friend-of-renoir,0,1,NaN,NaN
1,https://shakespeareandco.princeton.edu/members...,Berenice Abbott,"Abbott, Berenice",NaN,Female,False,False,1898.0,1991.0,1926,...,75007,7,"48.85681, 2.32644","Sylvia Beach’s <a href=""https://shakespeareand...",2020-09-22T15:43:11+00:00,abbott-berenice,0,1,NaN,NaN
2,https://shakespeareandco.princeton.edu/members...,Mrs. Paul Abbott,"Abbott, Mrs. Paul",Mrs.,Female,False,False,NaN,NaN,1925,...,75014,14,"48.83818, 2.33049",NaN,2020-01-24T18:12:40+00:00,abbott,0,1,NaN,NaN
3,https://shakespeareandco.princeton.edu/members...,Abdul,Abdul,NaN,NaN,False,False,NaN,NaN,1941,...,NaN,NaN,NaN,NaN,2019-12-22T01:57:37+00:00,abdul,0,1,NaN,NaN
4,https://shakespeareandco.princeton.edu/members...,Ernst Otto Abel,"Abel, Ernst Otto",Mr.,Male,False,False,NaN,NaN,1931;1932,...,75006,6,"48.85509, 2.33358","Sylvia Beach’s <a href=""https://shakespeareand...",2020-12-17T16:40:37+00:00,ernst-abel,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5596,https://shakespeareandco.princeton.edu/members...,Ziegler,Ziegler,NaN,NaN,False,False,NaN,NaN,1928,...,NaN,NaN,NaN,NaN,2019-11-19T14:47:39+00:00,ziegler,0,1,NaN,NaN
5597,https://shakespeareandco.princeton.edu/members...,Isabelle Zimmer,"Zimmer, Isabelle",Mlle,Female,False,True,1919.0,2001.0,1920;1921,...,75005;,5;,"48.84341, 2.33902;48.51081, -3.32637",NaN,2019-12-28T02:32:00+00:00,zimmer,0,4,overborrows,2
5598,https://shakespeareandco.princeton.edu/members...,H. Zongenth,"Zongenth, H.",Miss,Female,False,False,NaN,NaN,1926,...,NaN,NaN,NaN,NaN,2019-12-27T21:05:43+00:00,zongenth,0,1,NaN,NaN
5599,https://shakespeareandco.princeton.edu/members...,Miss Zueblin,"Zueblin, Miss",Miss,Female,False,False,NaN,NaN,1922,...,NaN,NaN,NaN,NaN,2019-12-27T01:44:22+00:00,zueblin,0,1,NaN,NaN


In [161]:
members_df[members_df.uri.duplicated()]

,uri,name,sort_name,title,gender,is_organization,has_card,birth_year,death_year,membership_years,...,postal_codes,arrondissements,coordinates,notes,updated,member_id,borrow_count,subscription_count,exceptional_types,exceptional_counts


In [162]:
books_df = pd.read_csv('SCoData_books_v1.1_2021-01_exceptional.csv')
books_df

,uri,title,author,translator,editor,contributor,illustrator,introduction,preface,photographer,...,volumes_issues,notes,event_count,borrow_count,purchase_count,circulation_years,updated,id,exceptional_type,counts
0,malory-morte-darthur,Le Morte d'Arthur,"Malory, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7,7,0,1937;1930;1939;1932;1938,2020-06-29T03:50:09+00:00,malory-morte-darthur,overborrows,2.0
1,more-utopia,Utopia,"More, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,2,0,1930;1950;1951,2020-04-30T19:40:14+00:00,more-utopia,overborrows,1.0
2,more-utopia,Utopia,"More, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,2,0,1930;1950;1951,2020-04-30T19:40:14+00:00,more-utopia,post1942_events,1.0
3,norton-gorboduc,Gorboduc,"Norton, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,0,1950,2020-06-29T03:49:47+00:00,norton-gorboduc,post1942_events,1.0
4,norton-gorboduc,Gorboduc,"Norton, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,0,1950,2020-06-29T03:49:47+00:00,norton-gorboduc,unknown_borrows,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6791,unclear-ile,[unclear]ile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Unidentified. Handwriting unclear.,1,1,0,NaN,2020-06-05T17:04:58+00:00,unclear-ile,NaN,NaN
6792,unclear-ism,[unclear]ism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Unidentified. Handwriting unclear.,1,1,0,1928,2020-06-02T22:08:10+00:00,unclear-ism,NaN,NaN
6793,unclear-4,[unclear]y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Unidentified. Handwriting unclear.,1,1,0,1926,2020-08-17T19:32:28+00:00,unclear-4,NaN,NaN
6794,unidentified-benn-s-library,[unidentified Benn’s Library],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Unidentified. From Benn's Sixpenny Library ser...,6,0,6,1931,2020-06-02T22:10:06+00:00,unidentified-benn-s-library,NaN,NaN


In [163]:
books_df[books_df.uri.duplicated()]

,uri,title,author,translator,editor,contributor,illustrator,introduction,preface,photographer,...,volumes_issues,notes,event_count,borrow_count,purchase_count,circulation_years,updated,id,exceptional_type,counts
2,more-utopia,Utopia,"More, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,2,0,1930;1950;1951,2020-04-30T19:40:14+00:00,more-utopia,post1942_events,1.0
4,norton-gorboduc,Gorboduc,"Norton, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,0,1950,2020-06-29T03:49:47+00:00,norton-gorboduc,unknown_borrows,1.0
18,browne-vulgar-errors-pseudodoxia,Vulgar Errors (Pseudodoxia Epidemica),"Browne, Thomas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,2,0,1939,2020-07-22T14:20:55+00:00,browne-vulgar-errors-pseudodoxia,sunday_shopers,1.0
22,milton-paradise-lost,Paradise Lost,"Milton, John",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6,6,0,1936;1939;1940,2020-05-13T17:04:32+00:00,milton-paradise-lost,unknown_borrows,1.0
27,swift-tale-tub,A Tale of a Tub,"Swift, Jonathan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6,5,1,1938;1939;1924;1941;1934,2020-05-16T17:39:20+00:00,swift-tale-tub,sunday_shopers,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6704,mackworth-time-tide,Time and Tide,NaN,NaN,"Mackworth, Margaret Haig Thomas, Viscountess R...",NaN,NaN,NaN,NaN,NaN,...,"Vol. 15, no. 45, Nov 10, 1934;Vol. 15, no. 46,...",NaN,118,109,7,1926;1933;1934;1935;1936;1937;1938,2020-12-02T16:23:44+00:00,mackworth-time-tide,unknown_borrows,7.0
6717,views-reviews,Views and Reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Unidentified. Henry James's *Views and Reviews...,4,4,0,1942;1939;1933;1941;1935,2020-06-02T20:09:41+00:00,views-reviews,sunday_shopers,1.0
6738,blake-blakes-works,Willam Blake's Works,"Blake, William",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Partial identification. At least three edition...,18,16,1,1921;1922;1923;1927;1929;1934;1936;1937;1938;1939,2020-06-03T01:35:29+00:00,blake-blakes-works,sunday_shopers,1.0
6739,blake-blakes-works,Willam Blake's Works,"Blake, William",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Partial identification. At least three edition...,18,16,1,1921;1922;1923;1927;1929;1934;1936;1937;1938;1939,2020-06-03T01:35:29+00:00,blake-blakes-works,unknown_borrows,1.0
